In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
)
!pip install evaluate
import evaluate
import numpy as np
import pandas as pd
from datasets import Dataset
!pip install sacrebleu
import sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.9 MB/s eta 0:00:00


In [2]:
# Load dataset using pandas
raw_df = pd.read_csv("/kaggle/input/english-nagamese-raw-data/raw_data.csv")
# raw_df = raw_df[:100]
raw_datasets = Dataset.from_pandas(raw_df)

# Split dataset
split_datasets = raw_datasets.train_test_split(train_size=0.8, seed=40)
split_datasets["validation"] = split_datasets.pop("test")

In [3]:
# Load tokenizer
model_checkpoint = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="ng_XX")

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [4]:
# Preprocessing function
max_length = 128
def preprocess_function(examples):
    inputs = examples["English"]
    targets = examples["Nagamese"]
    return tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)


Map:   0%|          | 0/6360 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [5]:
# Load model and move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [6]:
from transformers import EarlyStoppingCallback

# Define Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  
    save_strategy="epoch",      
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,   
    gradient_accumulation_steps=4, 
    warmup_steps=100, 
    weight_decay=0.01, 
    save_total_limit=4, 
    num_train_epochs=10, 
    learning_rate=5e-5, 
    predict_with_generate=True,  
    optim="adafactor", 
    report_to=[],  
    fp16=True, 
    logging_dir="./logs",
    logging_steps=100,  
    overwrite_output_dir=True,
    load_best_model_at_end=True, 
    metric_for_best_model="eval_loss",
    greater_is_better=False, 
)

# Trainer with Early Stopping
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Stops if no improvement for 3 evals
)

# Start Training
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-a84dcb066f87>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,5.636600,3.190588
2,4.658500,2.218999
3,1.897700,1.698579
4,1.470100,1.531140
5,1.218400,1.475219
6,1.041500,1.475461
7,0.905400,1.511274
8,0.798800,1.532428


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=800, training_loss=2.2033640861511232, metrics={'train_runtime': 2790.644, 'train_samples_per_second': 22.79, 'train_steps_per_second': 0.355, 'total_flos': 5951632501506048.0, 'train_loss': 2.2033640861511232, 'epoch': 8.0})

In [7]:
# Move model to evaluation mode
model.eval()

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=

In [8]:
# Extract source and reference texts from the validation set
source_texts = raw_df["English"].tolist()
reference_texts = raw_df["Nagamese"].tolist()

In [9]:
# Generate translations
machine_translations = []
for text in source_texts:
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    machine_translations.append(translated_text)

In [10]:
# Compute BLEU Score
# Metric computation
metric = evaluate.load("sacrebleu")
wrapped_reference_texts = [[ref] for ref in reference_texts] 
bleu = metric.compute(predictions=machine_translations, references=wrapped_reference_texts)
print("BLEU Score:", bleu["score"])

BLEU Score: 27.539482979211268
